In [220]:
#Imorting dependencies
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

In [221]:
import sqlalchemy
from sqlalchemy import create_engine

In [222]:
#Reading the Foreign Direct Investment Dataset
annual_inflation_file= "../Resources/Inflation_annual.csv"
annual_inflation_df = pd.read_csv( annual_inflation_file)
annual_inflation_df.head()

,"Inflation, GDP deflator (annual %)",1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,Abkhazia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.845357,6.780488,8.405298,2.413906,13.208274,19.643462,-1.162791,3.682878,NaN
2,Akrotiri and Dhekelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.300196,3.383486,6.007745,3.469252,1.995241,2.024081,4.360905,2.410882,3.459343,3.000000
4,Algeria,3.47172,2.35128,0.549331,1.695183,1.501331,1.817815,1.312041,3.142056,1.921084,...,1.906329,8.323803,10.629329,16.459258,11.282812,7.331055,14.602179,-11.266611,16.245617,11.431168


In [223]:
#Reading the Foreign Direct Investment Dataset
foreign_dir_inv_file= "../Resources/Foreign_direct_investment.csv"
foreign_dir_inv_df = pd.read_csv( foreign_dir_inv_file)
foreign_dir_inv_df.head()

,"Foreign direct investment, net inflows (% of GDP)",1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,Afghanistan,0.013151,0.024575,0.009401,0.015577,NaN,NaN,0.001565,-0.008126,NaN,...,1.152364,1.212725,3.276532,3.976666,3.082130,2.503252,2.512501,1.301564,0.438726,NaN
1,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.034135,3.149791,4.572142,3.133523,3.561523,6.186838,7.390890,8.008638,9.414123,NaN
2,Algeria,1.647378,0.011817,0.613595,0.585191,2.710127,0.764883,1.054808,0.850913,0.51263,...,1.866684,0.931657,1.037359,1.056292,1.532824,1.223827,1.517639,1.998263,1.414501,NaN
3,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [224]:
#Reading the Government Debt Dataset
govt_debt_file= "../Resources/Government_Debt.csv"
govt_debt_df = pd.read_csv( govt_debt_file)
govt_debt_df.head()

,COU,Country,DTYP,Type,FREQ,Frequency,UNIT,Unit,DVAR,Variable,TIME,Time period,Value,Flag Codes,Flags
0,AUS,Australia,AMT,Stocks: Outstanding amounts,A,Annual,PCT,Percentage,P1,Total central government debt % of GDP,1997,1997,18.476,NaN,NaN
1,AUS,Australia,AMT,Stocks: Outstanding amounts,A,Annual,PCT,Percentage,P1,Total central government debt % of GDP,1998,1998,15.578,NaN,NaN
2,AUS,Australia,AMT,Stocks: Outstanding amounts,A,Annual,PCT,Percentage,P1,Total central government debt % of GDP,1999,1999,13.741,NaN,NaN
3,AUS,Australia,AMT,Stocks: Outstanding amounts,A,Annual,PCT,Percentage,P1,Total central government debt % of GDP,2000,2000,11.361,NaN,NaN
4,AUS,Australia,AMT,Stocks: Outstanding amounts,A,Annual,PCT,Percentage,P1,Total central government debt % of GDP,2001,2001,9.552,NaN,NaN


Government Debt Data

Transform govt_debt_df so that it can be loaded into a table inside our postgresSQL database

In [225]:
# Select relevant columns needed
govt_debt_df_new = govt_debt_df[["Country","TIME","Value"]]
# Because the table isnt a format that we can use,  we can transform it via the pivot_table function
govt_debt_df_new_a = govt_debt_df_new.pivot_table('Value',['Country'],"TIME")
# We need to reset our index as transforming it via pivot_table has changed our index
govt_debt_df_new_a.reset_index(drop=False, inplace=True)
# Select the columns we are interested in storing (2001-2010)
govt_debt_df_new_b = govt_debt_df_new_a[['Country',2001,2002,2003,2004,2005,2006,2007,2008,2009,2010]]

In [226]:
# Set the axis to the columns we are keeping, this step is performed to remove the stacking caused via the pivot_table. This removed TIME as the stacked Key value.
govt_debt_df_new_b = govt_debt_df_new_b.set_axis(['Country',2001,2002,2003,2004,2005,2006,2007,2008,2009,2010], axis='columns')

In [227]:
# Fill Nan values with 0 so that we can import our data, the tables inside our postgresSQL DB is set up in a way that it will not accept null values.
govt_debt_df_new_b = govt_debt_df_new_b.fillna(0)
govt_debt_df_new_b.head()

,Country,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
0,Australia,9.552,8.576,7.549,6.716,6.312,5.760,5.181,4.922,8.195,10.966
1,Austria,60.659,60.401,60.882,62.242,62.116,60.434,57.829,59.319,64.916,65.754
2,Belgium,99.125,97.948,95.395,92.763,91.774,87.568,85.295,90.094,94.893,96.789
3,Canada,39.713,38.091,35.853,32.109,30.235,27.934,25.183,28.642,35.716,36.073
4,Chile,14.924,15.675,13.001,10.684,7.282,5.264,4.097,5.173,6.228,9.185


Foreign Direct Investment Data

Transform foreign_dir_inv_df so that it can be loaded into a table inside our postgresSQL database

In [228]:
# Select relevant columns needed
foreign_dir_inv_df=foreign_dir_inv_df[["Foreign direct investment, net inflows (% of GDP)","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010"]]
# Drop Nan Values so that the data can be imported into our postgresSQL DB, which is configured to not accept null values
foreign_dir_inv_df=foreign_dir_inv_df.dropna()
# Rename our columns
foreign_dir_inv_df.rename(columns = {'Foreign direct investment, net inflows (% of GDP)':'Country'}, inplace = True)

Set the index of the dataframe to "Country" as this will be a primary key inside our DB

In [229]:
foreign_dir_inv_df = foreign_dir_inv_df.set_index("Country")

In [230]:
foreign_dir_inv_df.head()

,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
Country,,,,,,,,,,
Afghanistan,0.027624,1.152364,1.212725,3.276532,3.976666,3.082130,2.503252,2.512501,1.301564,0.438726
Albania,5.067196,3.034135,3.149791,4.572142,3.133523,3.561523,6.186838,7.390890,8.008638,9.414123
Algeria,2.007757,1.866684,0.931657,1.037359,1.056292,1.532824,1.223827,1.517639,1.998263,1.414501
Angola,24.009226,14.626760,25.112026,7.328525,-4.256403,-0.083508,-1.477781,1.994536,2.921207,-3.799535
Antigua and Barbuda,12.399882,8.001069,19.192072,8.699671,21.594978,30.864149,25.662619,12.640130,6.435273,7.981280


Annual Inflation Data

Transform annual_inflation_df so that it can be loaded into a table inside our postgresSQL database

In [231]:
# Select relevant columns needed
annual_inflation_df=annual_inflation_df[["Inflation, GDP deflator (annual %)","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010"]]
# Drop Nan Values so that the data can be imported into our postgresSQL DB, which is configured to not accept null values
annual_inflation_df=annual_inflation_df.dropna()
# Rename our columns
annual_inflation_df.rename(columns = {'Inflation, GDP deflator (annual %)':'Country'}, inplace = True)

Set the index of the dataframe to "Country" as this will be a primary key inside our DB

In [232]:
annual_inflation_df = annual_inflation_df.set_index('Country')

In [233]:
annual_inflation_df.head()

,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
Country,,,,,,,,,,
Albania,3.458281,3.300196,3.383486,6.007745,3.469252,1.995241,2.024081,4.360905,2.410882,3.459343
Algeria,0.711210,1.906329,8.323803,10.629329,16.459258,11.282812,7.331055,14.602179,-11.266611,16.245617
Angola,108.480894,120.507256,102.538108,42.705463,25.954625,13.041180,12.616432,19.681609,-7.418838,22.393924
Antigua and Barbuda,-3.206067,1.129318,0.192628,-0.407634,6.310777,0.302848,23.258314,3.021693,-0.013833,4.280006
Argentina,-1.095779,30.555282,10.495608,9.220651,8.840490,13.426278,14.258239,19.068450,9.977458,15.376172


Create Tables inside of PGAdmin (PostgresSQL Server)

Connect to postgresSQL server

In [234]:
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'econ_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

Look at the tables available inside the SQL Database we have connected to.

In [235]:
insp.get_table_names()

['Inflation', 'Foreign_Investment', 'Government_Debt', 'freshwater']

Load our annual_inflation_df Dataframe into the table 'Inflation' on our live SQL database

In [236]:
annual_inflation_df.to_sql(name='Inflation', con=engine, if_exists='append')

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "Inflation_pkey"
DETAIL:  Key ("Country")=(Albania) already exists.

[SQL: INSERT INTO "Inflation" ("Country", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010") VALUES (%(Country)s, %(2001)s, %(2002)s, %(2003)s, %(2004)s, %(2005)s, %(2006)s, %(2007)s, %(2008)s, %(2009)s, %(2010)s)]
[parameters: ({'Country': 'Albania', '2001': 3.458280955, '2002': 3.3001958719999998, '2003': 3.38348607, '2004': 6.007744671, '2005': 3.4692522000000006, '2006': 1.995241221, '2007': 2.024081357, '2008': 4.360904563, '2009': 2.4108820680000003, '2010': 3.45934259}, {'Country': 'Algeria', '2001': 0.7112095184999999, '2002': 1.906328849, '2003': 8.323802693, '2004': 10.62932921, '2005': 16.45925846, '2006': 11.28281156, '2007': 7.3310551870000005, '2008': 14.60217944, '2009': -11.2666112, '2010': 16.24561679}, {'Country': 'Angola', '2001': 108.4808937, '2002': 120.5072557, '2003': 102

Load our foreign_dir_inv_df Dataframe into the table 'Foreign_Investment' on our live SQL database

In [ ]:
foreign_dir_inv_df.to_sql(name='Foreign_Investment', con=engine, if_exists='append')

Load our govt_debt_df_new_b Dataframe into the table 'Government_Debt' on our live SQL database

In [ ]:
govt_debt_df_new_b.to_sql(name='Government_Debt', con=engine, if_exists='append')